<a href="https://colab.research.google.com/github/Ganeshs170220/AI-MODELS/blob/main/SUMMARIZATION_FROM_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This model is created using hugging face transformers and beautifulsoup

#installing required modules

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00


In [3]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=b4f77e5bb57e0648318fcf53cfa7fd36757e77b2f53c74b222c18fa47ba5996d
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


#import the modules

In [4]:
from transformers import pipeline
import requests
from bs4 import BeautifulSoup

In [5]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


#read the url from google

In [6]:
URL = ("https://en.wikipedia.org/wiki/Narendra_Modi")
r = requests.get(URL)

#parsing the data from that url

In [7]:
soup = BeautifulSoup(r.text,'html.parser')
results = soup.find_all(['h1','p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

#printing the parsed data from the URL

In [8]:
ARTICLE

'Narendra Modi \n Premiership\n Budgets\n \n Campaigns\n Missions\n Establishments and foundations\n Events and observances\n Military and rescue operations\n Treaties and accords\n \n Establishments and foundations\n Events and observances\n Military and rescue operations\n Treaties and accords\n Controversies\n \n Narendra Damodardas Modi (Gujarati:\xa0[ˈnəɾendɾə dɑmodəɾˈdɑs ˈmodiː] (listen); born 17 September 1950)[b] is an Indian politician who has served as the 14th Prime Minister of India since May 2014. Modi was the Chief Minister of Gujarat from 2001 to 2014 and is the Member of Parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation. He is the longest-serving prime minister from outside the Indian National Congress.\n Modi was born and raised in Vadnagar in northeastern Gujarat, where he completed his secondary education. He was introduced t

#Taking a chunk size

In [9]:
max_chunk = 500

#Data preprossing


In [10]:
ARTICLE = ARTICLE.replace('\n','')
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

#splitting the data and creating chunks

In [11]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [12]:
len(chunks)

22

#summarize the each and every chunk

In [13]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

#join the all chunks and getting the final summary

In [14]:
final_summary = ' '.join([summ['summary_text'] for summ in res])

In [15]:
final_summary

' Narendra Modi is the 14th Prime Minister of India since May 2014 . He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS) He is the longest-serving prime minister from outside the Indian National Congress . Modi was the Chief Minister of Gujarat from 2001 to 2014 and is the Member of Parliament (MP) for Varanasi .  Narendra Damodardas Modi was born on 17 September 1950 to a Gujarati Hindu family of grocers in Vadnagar, Mehsana district, Bombay State (present-day Gujarat). He was introduced to the Rashtriya Swayamsevak Sangh (RSS) and began attending local shakhas (training sessions) when he was eight years old . Modi has been described as engineering a political realignment towards right-wing politics . He remains a controversial figure domestically and internationally .  Narendra Modi\'s family arranged a betrothal to Jashodaben Chimanlal Modi, leading to their marriage when she was 17 and he was 18 . Soon afterwards, he abandoned his wife an